In [ ]:
from re import sub
from typing import cast
import geopandas as gpd
import pandas as pd
import numpy as np
# import pmatcher
from tqdm import tqdm
import re
from functools import partial
import ast 

In [ ]:
Pgov18 = pd.read_csv("/content/drive/MyDrive/MGGG/WI_VRA/2018 elections/Pgov18.csv")
Plg18 = pd.read_csv("/content/drive/MyDrive/MGGG/WI_VRA/2018 elections/Plg18.csv")
Psos18 = pd.read_csv("/content/drive/MyDrive/MGGG/WI_VRA/2018 elections/Psos18.csv")
Ptres18 = pd.read_csv("/content/drive/MyDrive/MGGG/WI_VRA/2018 elections/Ptres18.csv")

In [ ]:
def string_standarize(s):
    s = s.replace(" - ", "-")
    s = s.replace(" & ", ",")
    s = s.replace("&", ",")
    s = s.replace(" AND ", ",")
    s = s.replace(", COMBINED", "")
    return s

In [ ]:
Pgov18["WARDGROUP"] = Pgov18["COUNTY"]  + ", " + Pgov18["WARD GROUP"].apply(string_standarize)
Plg18["WARDGROUP"] = Plg18["COUNTY"]  + ", " + Plg18["WARD GROUP"].apply(string_standarize)
Psos18["WARDGROUP"] = Psos18["COUNTY"]  + ", " + Psos18["WARD GROUP"].apply(string_standarize)
Ptres18["WARDGROUP"] = Ptres18["COUNTY"]  + ", " + Ptres18["WARD GROUP"].apply(string_standarize)

In [ ]:
def str_range_to_list(s):
    contains_alpha = lambda s: "A" in s or "B" in s or "C" in s or "S" in s
    nur_numeric = lambda s: not contains_alpha(s)

    if contains_alpha(s):
         ranges = s.split(",")
         alphas = list(filter(contains_alpha, ranges))
         non_alphas = list(filter(nur_numeric, ranges))
         ranges = [x.split("-") for x in non_alphas]
         return alphas + [str(i) for r in ranges for i in range(int(r[0]), int(r[-1]) + 1)]
    else:
        ranges = (x.split("-") for x in s.split(","))
        return [str(i) for r in ranges for i in range(int(r[0]), int(r[-1]) + 1)]

def str_range_to_list_wrapper(s):
    try:
        return str_range_to_list(s)
    except:
        return None

In [ ]:
ward_names = " WARD | WARDS | WD | WARD| WARDS-| WARDS| WDS | WARD-"

In [ ]:
for df in [Pgov18,Plg18,Psos18,Ptres18]:
    df["ward_jurisdiction"] = df.WARDGROUP.apply(lambda p: re.split(ward_names, p)[0])
    df["ward_list_str"] = df.WARDGROUP.apply(lambda p: re.split(ward_names, p)[-1])

In [ ]:
for df in [Pgov18,Plg18,Psos18,Ptres18]:
    df["ward_list"] = df["ward_list_str"].apply(str_range_to_list_wrapper)

In [ ]:
Pgov18[Pgov18["ward_list"] == None]
Plg18[Plg18["ward_list"] == None]
Psos18[Psos18["ward_list"] == None]
Ptres18[Ptres18["ward_list"] == None]

,COUNTY,WARD GROUP,TOTAL,DAWN MARIE SASS,CYNTHIA KAUMP,SARAH GODLEWSKI,WARDGROUP,ward_jurisdiction,ward_list_str,ward_list


In [ ]:
Pgov18.to_csv("Pgov18_parsed.csv", index=False)
Plg18.to_csv("Plg18_parsed.csv", index=False) # 3880 rows - Unique by county + wardgroup
Psos18.to_csv("Psos18_parsed.csv", index=False)
Ptres18.to_csv("Ptres18_parsed.csv", index=False)